In [1]:
# On importe la librairie pandas puis on spécifie le chemin d'accès des 4 fichiers csv
import pandas as pd
import numpy as np
import functions as fc
import imlconnect
from datetime import datetime, timedelta
import time
start = time.process_time()

In [2]:
print("Entrez la date du jour svp ?\n")
j = input("Jour ? (format numérique svp!)\n")
while len(j)!=2:
    j = input("Jour ? (2 caractères et format numérique svp!)\n")
while True:
    try:
        val = int(str(j))
        while val<1 or val>31:
            j = input("Jour ? Entre 01 et 31 svp!)\n") 
            val = int(str(j))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        j = input("Jour ? (format numérique svp!)\n")
    else:
        print("Succès !\n")
        # No error; stop the loop
        break  

m = input("Mois ? (format numérique svp!)\n")
while len(m)!=2:
    m = input("Mois ? (2 caractères et format numérique svp!)\n")
while True:
    try:
        val1 = int(str(m))
        while val1<1 or val1>12:
            m = input("Mois ? Entre 01 et 12 svp!)\n") 
            val1 = int(str(m))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        m = input("Mois ? (format numérique svp!)\n")
    else:
        print("Succès !\n")# No error; stop the loop
        break

a = input("Année en cours? (format numérique svp!)\n")
while len(a)!=4:
    a = input("Année ? (4 caractères et format numérique svp!)\n")
while True:
    try:
        val2 = int(str(a))
        while val2!=datetime.now().year:
            a = input("Année doit être année en cours svp!)\n") 
            val2 = int(str(a))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        a = input("Année ? (format numérique svp!)\n")
    else:
        print("Succès !\n")# No error; stop the loop
        break

Entrez la date du jour svp ?



Jour ? (format numérique svp!)
 29


Succès !



Mois ? (format numérique svp!)
 12


Succès !



Année en cours? (format numérique svp!)
 2022


Succès !



# Utiliser j-2 si les données j-1 ne sont pas présentes

In [3]:
date_saisie = pd.to_datetime(str(val2)+"-"+str(val1)+"-"+str(val)).strftime("%Y-%m-%d")
print("On va récupérer les données dans LinkyPilot en fonction de l'index date (J-1), vous devez retirer 1 jour à la date saisie :\n")
print("Si les résultats ne sont pas satisfaisants, relancez ce script en retirant 2 jours (J-2).\n")
Q = input("Voulez retirer des jours à la date d'analyse ? oui ou non ?\n")
while Q!='oui' and Q!='non':
    print("Ecrivez correctement oui ou non en minuscule svp!\n")
    Q = input("Voulez retirer des jours à la date d'analyse ? oui ou non ?\n")
if Q=="oui":
    jour_à_retirer = input("Entrer le nombre de jour à retirer svp ?\n")
    while True:
        try:
            jour_à_retirer = int(str(jour_à_retirer))
        except ValueError:
            print("Ce n'est pas un nombre entier\n")
            jour_à_retirer = input("Entrez un nombre entier svp ?\n")
        else:
            print("Succès !\n")
            break
    date_analyse = pd.to_datetime(date_saisie) - timedelta(days=jour_à_retirer)
    date_analyse = date_analyse.strftime("%Y-%m-%d")
    print("La date d'analyse est : \n",date_analyse)             
elif Q=='non':
    date_analyse = pd.to_datetime(date_saisie) - timedelta(days=0)
    date_analyse = date_analyse.strftime("%Y-%m-%d")
    
print("La date d'analyse est : \n",date_analyse) 
print("La date saisie est : \n",date_saisie)
print("Veuillez attendre jusqu'à la fin du traitement svp !\n")    

On va récupérer les données dans LinkyPilot en fonction de l'index date (J-1), vous devez retirer 1 jour à la date saisie :

Si les résultats ne sont pas satisfaisants, relancez ce script en retirant 2 jours (J-2).



Voulez retirer des jours à la date d'analyse ? oui ou non ?
 oui
Entrer le nombre de jour à retirer svp ?
 1


Succès !

La date d'analyse est : 
 2022-12-28
La date d'analyse est : 
 2022-12-28
La date saisie est : 
 2022-12-29
Veuillez attendre jusqu'à la fin du traitement svp !



In [4]:
#LinkyCoeur
my_conn_rcm = imlconnect.Sql("rcm_conn", 
                      key_path='/var/projects/iml/rstudio/D86553/python_conn/imlconnect.key', 
                      config_file_path='/var/projects/iml/rstudio/D86553/python_conn/config_imlconnect.yml',
                      open_connection=True)

In [5]:
df_ECART_POSE_DECOUVERTE = my_conn_rcm.query("""select x.ID_C, x.prm_dec, y.prm_pose, y.id_pdc from (select /*+parallel(8)*/ lpad(id_c,12,'0') id_c,
                                       max(id_prm) keep (dense_rank first order by date_debut desc, date_maj desc) prm_dec
                                       from rcm.c_prm group by id_c) x
                                       inner join (select a.id_pdc, id_c , id_prm_conso prm_pose
                                       from rcm.cpdc_lien_courant a 
                                       inner join rcm.pdc_pdc b  on a.id_pdc = b.id_pdc where STATUT_LIEN_C_PDC = 2) y 
                                       on x.id_c = y.id_c and x.prm_dec<>y.prm_pose""")


In [6]:
sql_ini = "select PDC_ID as ID_PDC, DIS_ID,DIS_LIBELLE DISTRIBUTEUR, PDC_ID_PRM_CONSO PRM_POSE_SUP from lspmet.t_pdc inner join filter_list f on f.val = pdc_id inner join  lspmet.t_ref_distributeur on pdc_distributeur_fk=dis_id where pdc_statut_fk=3"
sql = fc.create_sql_with_df(df_ECART_POSE_DECOUVERTE,'id_pdc',sql_ini)


In [7]:
my_conn_sup = imlconnect.Sql("lsup_conn", 
                      key_path='/var/projects/iml/rstudio/D86553/python_conn/imlconnect.key', 
                      config_file_path='/var/projects/iml/rstudio/D86553/python_conn/config_imlconnect.yml',
                      open_connection=True)

In [8]:
df_T_PDC_RECO = my_conn_sup.query(sql)

In [9]:

df_ECART_POSE_DECOUVERTE = df_ECART_POSE_DECOUVERTE[df_ECART_POSE_DECOUVERTE.id_pdc!='PDC00016543785'].merge(df_T_PDC_RECO.rename(columns = {'prm_pose_sup':'prm_pose'}),on = ['id_pdc','prm_pose'],how = 'inner')

In [10]:
# le .merge sert à fusionner 2 df (jointure), il peut se faire sur 1 ou plusieurs colonne la condition étant que les colonnes qui servent d'index de fusion doivent avoir le même nom, le parametre indicator = True indique lors d'une jointure externe ('outer'), 
# avec la colonne _merge, s'il s'agit des lignes de la table de gauche, celle de droite ou des 2, cette option sert lorsqu'on a besoin de voir les lignes communnes aux 2 df ou non
# J'AI CREE CE TYPE DE JOINTURE CAR PARFOIS ON PEUT AVOIR BESOIN DE REGARDER CE QU'ON fait des lignes de non-correspondance lors d'une jointure interne ('inner'), car dans notre exemple on fait un inner join de df_ECART_POSE_DECOUVERTE et df_T_PDC_RECO
df_ECART_POSE_DECOUVERTE_temp = df_ECART_POSE_DECOUVERTE.merge(df_T_PDC_RECO.rename(columns = {'prm_pose_sup':'prm_pose'}),on =['id_pdc','prm_pose'], how = 'outer',indicator = True)
df_ECART_POSE_DECOUVERTE_temp

id_c         prm_dec        prm_pose          id_pdc  dis_id_x  \
0      021776655465  76127209110131  76103190210186  PDC00013859177       9.0   
1      021861113314  50099945094181  19574095501446  PDC00015978809       2.0   
2      021875224452  06392908766824  06392185177860  PDC00012148319       2.0   
3      022076337342  50066219350231  15556005725434  PDC00029739190       2.0   
4      022076572838  00000000000000  22103038911710  PDC00019790955       2.0   
...             ...             ...             ...             ...       ...   
13653  031864278903  00000000000000  75711158210107  PDC00018400806       8.0   
13654  041875011825  00000000000000  01636324076668  PDC00022266560       2.0   
13655           NaN             NaN  76213379310180  PDC00012580359       NaN   
13656           NaN             NaN  76213347510150  PDC00014367580       NaN   
13657           NaN             NaN  14357452959376  PDC00016543785       NaN   

      distributeur_x  dis_id_y distributeur_y      _merge  
0            SEI_GUA         9        SEI_GUA        both  
1             ENEDIS         2         ENEDIS        both  
2             ENEDIS         2         ENEDIS        both  
3             ENEDIS         2         ENEDIS        both  
4             ENEDIS         2         ENEDIS        both  
...              ...       ...            ...         ...  
13653        SEI_COR         8        SEI_COR        both  
13654         ENEDIS         2         ENEDIS        both  
13655            NaN         6        SEI_MAR  right_only  
13656            NaN         6        SEI_MAR  right_only  
13657            NaN         2         ENEDIS  right_only  

[13658 rows x 9 columns]

In [11]:
not_correspondance_left = df_ECART_POSE_DECOUVERTE_temp[df_ECART_POSE_DECOUVERTE_temp['_merge']=='left_only']
not_correspondance_left

Empty DataFrame
Columns: [id_c, prm_dec, prm_pose, id_pdc, dis_id_x, distributeur_x, dis_id_y, distributeur_y, _merge]
Index: []

In [12]:
not_correspondance_right = df_ECART_POSE_DECOUVERTE_temp[df_ECART_POSE_DECOUVERTE_temp['_merge']=='right_only']
not_correspondance_right

id_c prm_dec        prm_pose          id_pdc  dis_id_x distributeur_x  \
13655  NaN     NaN  76213379310180  PDC00012580359       NaN            NaN   
13656  NaN     NaN  76213347510150  PDC00014367580       NaN            NaN   
13657  NaN     NaN  14357452959376  PDC00016543785       NaN            NaN   

       dis_id_y distributeur_y      _merge  
13655         6        SEI_MAR  right_only  
13656         6        SEI_MAR  right_only  
13657         2         ENEDIS  right_only

In [13]:
df_T_DA_ECART_PRM =   my_conn_sup.query("""select PDC_ID ,  DAN_NNI_OUVERTURE, DA.DAN_NUMERO_DOSSIER NUMERO_DA, TDY.TDY_LIBELLE NOM_DA, statut_da.sda_libelle STATUT_DA
                                       from lspmet.t_dossier_analyse DA
                                       join lspmet.T_ref_statut_dossier_analyse statut_da on DA.DAN_STATUT_DOSSIER_ANALYSE_FK = statut_da.sda_id
                                       join lspmet.t_dysfonctionnement DYS on DYS.DYS_ID= DA.DAN_DYSFONCTIONNEMENT_FK
                                       JOIN lspmet.T_REF_TYPE_DYSFONCTIONNEMENT TDY ON DYS.DYS_TYPE_DYSFONCTIONNEMENT_FK = TDY.TDY_ID
                                       JOIN lspmet.t_pdc on dys_id_objet = pdc_id
                                       WHERE TDY_ID in (33)""")

In [14]:
sql_ini1 = "select /*+parallel(8)*/ lpad(id_c,12,'0') id_c, max(id_prm) keep (dense_rank first order by date_debut asc, date_maj asc) PRM_PREMIERE_DECOUVERTE from rcm.c_prm inner join filter_list f on f.val = c_prm.id_c group by id_c"
sql1 = fc.create_sql_with_df(df_ECART_POSE_DECOUVERTE,'id_c',sql_ini1)

In [15]:
# Recherche du PRM de toute première découverte du C d'après la table C_PRM de LINKYCOEUR
# A remplacer table_temp par my_char
df_T_DECOUVERTE_C =  my_conn_rcm.query(sql1)

In [16]:
df_T_DECOUVERTE_C

id_c prm_premiere_decouverte
0      021570018165          01537047723028
1      021575004976          14270188003290
2      021575005271          00000000000000
3      021575016753          00000000000000
4      021662001788          17592329935559
...             ...                     ...
13650  841875524840          00000000000000
13651  841875547796          00000000000000
13652  841875549662          25378726298071
13653  851764000017          99999999500002
13654  851764010767          19961215611799

[13655 rows x 2 columns]

In [17]:
nom="langage"
'**'.join(nom)+"**"

'l**a**n**g**a**g**e**'

In [18]:
df_T_DA_ECART_PRM = df_T_DA_ECART_PRM.assign(Nb_Da = 1)

In [19]:
df_T_DA_ECART_PRM_groupé = df_T_DA_ECART_PRM.groupby(by=["pdc_id","statut_da"],as_index=False).Nb_Da.sum()

In [20]:
df_DERNIER_DA = df_T_DA_ECART_PRM.groupby(by="pdc_id",as_index=False).Nb_Da.sum().merge(df_T_DA_ECART_PRM.drop('Nb_Da',axis=1).sort_values(by="numero_da",ascending = False),on="pdc_id",how = 'inner')

In [21]:
# Statistiques 2
df_NB_DA_STATS = df_T_DA_ECART_PRM_groupé.pivot_table(index = "pdc_id", columns = "statut_da",values = "Nb_Da") 
df_NB_DA_STATS = df_NB_DA_STATS.reset_index().rename_axis(None, axis = 1)


In [22]:
df_NB_DA_STATS.isnull().sum()

pdc_id           0
Clos         12644
En cours    730143
dtype: int64

In [23]:
df_NB_DA_STATS["En cours"].value_counts()

1.0    13358
Name: En cours, dtype: int64

In [24]:
df_NB_DA_STATS = df_NB_DA_STATS.replace(np.nan,0).rename(columns = {"Clos":"Nb_Da_Clos","En cours":"Nb_Da_en_cours"})

In [25]:
# Affiche le nombre total de valeur null d'une colonne : 
# syntaxe: nom_df.nom_col.isnull().sum()
df_NB_DA_STATS.Nb_Da_Clos.isnull().sum()

0

In [26]:
# sans le .sum(), il affiche une colonne de booléen (True si null, False si non-null)
df_NB_DA_STATS

pdc_id  Nb_Da_Clos  Nb_Da_en_cours
0       PDC00000000339         1.0             0.0
1       PDC00000000365         1.0             0.0
2       PDC00000000693         1.0             0.0
3       PDC00000000797         1.0             0.0
4       PDC00000000981         1.0             0.0
...                ...         ...             ...
743496  PSC00030758507         1.0             0.0
743497  PSC00030759083         1.0             0.0
743498  PSC00030759146         1.0             0.0
743499  PSC00030759614         1.0             0.0
743500  PSC00036927810         3.0             0.0

[743501 rows x 3 columns]

In [27]:

df_Analyse_Ecart_PRM_BILAN = pd.merge(df_ECART_POSE_DECOUVERTE, df_T_DECOUVERTE_C,on='id_c',how='left').merge(df_DERNIER_DA.rename(columns={'pdc_id':'id_pdc'}),on='id_pdc',how='left').merge(df_NB_DA_STATS.rename(columns={'pdc_id':'id_pdc'}),on='id_pdc',how='left')


In [28]:
df_Analyse_Ecart_PRM_BILAN = df_Analyse_Ecart_PRM_BILAN.replace({'Nb_Da_Clos': np.nan, 'Nb_Da_en_cours': np.nan}, 0)

In [29]:
# Je vérifie bien qu'il n'y a plus de valeur null, c'est le cas
print(df_Analyse_Ecart_PRM_BILAN.Nb_Da_Clos.isnull().sum())
print(df_Analyse_Ecart_PRM_BILAN.Nb_Da_en_cours.isnull().sum())

0
0


In [30]:
df_Analyse_Ecart_PRM_BILAN

id_c         prm_dec        prm_pose          id_pdc  dis_id  \
0      021776655465  76127209110131  76103190210186  PDC00013859177       9   
1      021861113314  50099945094181  19574095501446  PDC00015978809       2   
2      021875224452  06392908766824  06392185177860  PDC00012148319       2   
3      022076337342  50066219350231  15556005725434  PDC00029739190       2   
4      022076572838  00000000000000  22103038911710  PDC00019790955       2   
...             ...             ...             ...             ...     ...   
14749  022164051649  00000000000000  23100000000210  PDC00042796738       2   
14750  022264026369  00000000000000  50089522853113  PDC00042828240       2   
14751  811961908959  07103473180172  07103328462306  PDC00027786149       2   
14752  031864278903  00000000000000  75711158210107  PDC00018400806       8   
14753  041875011825  00000000000000  01636324076668  PDC00022266560       2   

      distributeur prm_premiere_decouverte  Nb_Da dan_nni_ouverture  \
0          SEI_GUA          76103190210186    1.0            D86553   
1           ENEDIS          19574095501446    1.0            B28746   
2           ENEDIS          06392185177860    1.0            E30774   
3           ENEDIS          15556005725434    1.0            B28746   
4           ENEDIS          00000000000000    1.0              None   
...            ...                     ...    ...               ...   
14749       ENEDIS          00000000000000    1.0              None   
14750       ENEDIS          00000000000000    1.0              None   
14751       ENEDIS          07103328462306    1.0            B28746   
14752      SEI_COR          00000000000000    1.0              None   
14753       ENEDIS          00000000000000    1.0              None   

            numero_da                 nom_da statut_da  Nb_Da_Clos  \
0      DA210720EXYGIV  Réco C - Écart de PRM  En cours         0.0   
1      DA211115FCTTVZ  Réco C - Écart de PRM  En cours         0.0   
2      DA210818EZDENA  Réco C - Écart de PRM  En cours         0.0   
3      DA221017FLNQZX  Réco C - Écart de PRM  En cours         0.0   
4      DA221223FMWWZX  Réco C - Écart de PRM  En cours         0.0   
...               ...                    ...       ...         ...   
14749  DA221209FMPAUY  Réco C - Écart de PRM  En cours         0.0   
14750  DA221223FMWXDH  Réco C - Écart de PRM  En cours         0.0   
14751  DA211115FCTTXU  Réco C - Écart de PRM  En cours         0.0   
14752  DA221024FLRAJQ  Réco C - Écart de PRM  En cours         0.0   
14753  DA190920DBLKKS  Réco C - Écart de PRM  En cours         0.0   

       Nb_Da_en_cours  
0                 1.0  
1                 1.0  
2                 1.0  
3                 1.0  
4                 1.0  
...               ...  
14749             1.0  
14750             1.0  
14751             1.0  
14752             1.0  
14753             1.0  

[14754 rows x 14 columns]

In [31]:
df_Analyse_Ecart_PRM_BILAN.Nb_Da_en_cours.value_counts()

1.0    14019
0.0      735
Name: Nb_Da_en_cours, dtype: int64

In [32]:
df_eligibles = df_Analyse_Ecart_PRM_BILAN[df_Analyse_Ecart_PRM_BILAN.Nb_Da_en_cours == 0].reset_index(drop=True)

In [33]:
df_eligibles

id_c         prm_dec        prm_pose          id_pdc  dis_id  \
0    031764170375  00000000000000  22508972405040  PDC00005180919       2   
1    022064781761  00000000000000  22597829097970  PDC00005740924       2   
2    022064781761  00000000000000  22597829097970  PDC00005740924       2   
3    021876728931  00000000000000  02313892857902  PDC00013471512       2   
4    031776031234  00000000000000  22483936322502  PDC00006545185       2   
..            ...             ...             ...             ...     ...   
730  841775198605  23583068008684  99999993000002  PDC00008816358       2   
731  031764116805  00000000000000  22476121449802  PDC00005140797       2   
732  031764116805  00000000000000  22476121449802  PDC00005140797       2   
733  031864292687  00000000000000  76202591310127  PDC00018400689       6   
734  031764067572  00000000000000  19169753961157  PDC00004562617       2   

    distributeur prm_premiere_decouverte  Nb_Da dan_nni_ouverture  \
0         ENEDIS          00000000000000    1.0          CP2D08DN   
1         ENEDIS          00000000000000    2.0          CP2D08DN   
2         ENEDIS          00000000000000    2.0              None   
3         ENEDIS          00000000000000    1.0              None   
4         ENEDIS          22483936322502    1.0          CP2D08DN   
..           ...                     ...    ...               ...   
730       ENEDIS          00000000000000    2.0              None   
731       ENEDIS          00000000000000    2.0            F40944   
732       ENEDIS          00000000000000    2.0              None   
733      SEI_MAR          00000000000000    1.0              None   
734       ENEDIS          19169753961157    1.0          CP2D08DN   

          numero_da                 nom_da statut_da  Nb_Da_Clos  \
0    DA201126EKVEIT  Réco C - Écart de PRM      Clos         1.0   
1    DA201126EKVEVG  Réco C - Écart de PRM      Clos         2.0   
2    DA201027EINXMY  Réco C - Écart de PRM      Clos         2.0   
3    DA180809BIBGRD  Réco C - Écart de PRM      Clos         1.0   
4    DA201125EKRIPB  Réco C - Écart de PRM      Clos         1.0   
..              ...                    ...       ...         ...   
730  DA180105ALEUHI  Réco C - Écart de PRM      Clos         2.0   
731  DA210921FAMOCD  Réco C - Écart de PRM      Clos         2.0   
732  DA210808EYRUJZ  Réco C - Écart de PRM      Clos         2.0   
733  DA210401ESSEWV  Réco C - Écart de PRM      Clos         1.0   
734  DA201125EKRFUU  Réco C - Écart de PRM      Clos         1.0   

     Nb_Da_en_cours  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  
..              ...  
730             0.0  
731             0.0  
732             0.0  
733             0.0  
734             0.0  

[735 rows x 14 columns]

In [34]:
my_conn_pilot = imlconnect.Sql("pilot_conn", 
                      key_path='/var/projects/iml/rstudio/D86553/python_conn/imlconnect.key', 
                      config_file_path='/var/projects/iml/rstudio/D86553/python_conn/config_imlconnect.yml',
                      open_connection=True)
        

In [35]:
sql_ini2 = "select ID_PDC, ID_PDK_COM, NB_JOUR_NON_COLLECTE from lptdwh_dmt.FAIT_PDC FPDC inner join filter_list f on f.val = FPDC.id_pdc where id_jour=to_date('"+ date_analyse + "','yyyy-mm-dd')"
sql2 = fc.create_sql_with_df(df_eligibles,'id_pdc',sql_ini2)

In [36]:

# Rechercher dans Pilot le nombre de jour de non collecte
df_MyData_Collecte = my_conn_pilot.query(sql2)

In [37]:
df_Analyse_Ecart_PRM_BILAN.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14754 entries, 0 to 14753
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_c                     14754 non-null  object 
 1   prm_dec                  14754 non-null  object 
 2   prm_pose                 14754 non-null  object 
 3   id_pdc                   14754 non-null  object 
 4   dis_id                   14754 non-null  int64  
 5   distributeur             14754 non-null  object 
 6   prm_premiere_decouverte  14754 non-null  object 
 7   Nb_Da                    14613 non-null  float64
 8   dan_nni_ouverture        3228 non-null   object 
 9   numero_da                14613 non-null  object 
 10  nom_da                   14613 non-null  object 
 11  statut_da                14613 non-null  object 
 12  Nb_Da_Clos               14754 non-null  float64
 13  Nb_Da_en_cours           14754 non-null  float64
dtypes: float64(3), int64(1

In [38]:
df_MyData_Collecte.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_pdc                533 non-null    object
 1   id_pdk_com            533 non-null    object
 2   nb_jour_non_collecte  533 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 12.6+ KB


In [39]:
# Ne conserver que les collectant et exclure les PINKY et les PRM 000 999 et PDK 69993
df_Ecart_PRM_TrouRaquette = df_Analyse_Ecart_PRM_BILAN.merge(df_MyData_Collecte,on='id_pdc',how = 'inner').query("nb_jour_non_collecte<=3")

In [40]:
df_Ecart_PRM_TrouRaquette = df_Ecart_PRM_TrouRaquette[~df_Ecart_PRM_TrouRaquette.prm_pose.str[0:4].isin(['9999','0000'])]
df_Ecart_PRM_TrouRaquette

id_c         prm_dec        prm_pose          id_pdc  dis_id  \
8    021875130987  00000000000000  22554558557460  PDC00010497943       2   
25   062064347901  1053B29                1053B29  PDC00028516624       2   
26   062064347901  1053B29                1053B29  PDC00028516624       2   
27   062064347901  1053B29                1053B29  PDC00028516624       2   
28   062064347901  1053B29                1053B29  PDC00028516624       2   
..            ...             ...             ...             ...     ...   
691  062064347915  2146B06                2146B06  PDC00028514451       2   
692  062064347915  2146B06                2146B06  PDC00028514451       2   
693  062064347915  2146B06                2146B06  PDC00028514451       2   
696  042113000218  LGDO56114P0047  LGDO56114R0047  PDC00042587376       2   
697  021875042319  01106078079395  01105788643761  PDC00012518035       2   

    distributeur prm_premiere_decouverte  Nb_Da dan_nni_ouverture  \
8         ENEDIS          22554558557460    NaN               NaN   
25        ENEDIS          00000000000000    6.0            I09772   
26        ENEDIS          00000000000000    6.0            F17575   
27        ENEDIS          00000000000000    6.0            B28746   
28        ENEDIS          00000000000000    6.0            B28746   
..           ...                     ...    ...               ...   
691       ENEDIS          00000000000000    4.0            B28746   
692       ENEDIS          00000000000000    4.0          CMBAA65N   
693       ENEDIS          00000000000000    4.0              None   
696       ENEDIS          00000000000000    1.0              None   
697       ENEDIS          01105788643761    NaN               NaN   

          numero_da                 nom_da statut_da  Nb_Da_Clos  \
8               NaN                    NaN       NaN         0.0   
25   DA221114FLZWGU  Réco C - Écart de PRM      Clos         6.0   
26   DA221031FLUIEU  Réco C - Écart de PRM      Clos         6.0   
27   DA221017FLNRFV  Réco C - Écart de PRM      Clos         6.0   
28   DA211214FDVSUG  Réco C - Écart de PRM      Clos         6.0   
..              ...                    ...       ...         ...   
691  DA211214FDVSTV  Réco C - Écart de PRM      Clos         4.0   
692  DA211103FCKAQZ  Réco C - Écart de PRM      Clos         4.0   
693  DA200907EETTHZ  Réco C - Écart de PRM      Clos         4.0   
696  DA220912FKQSOA  Réco C - Écart de PRM      Clos         1.0   
697             NaN                    NaN       NaN         0.0   

     Nb_Da_en_cours        id_pdk_com  nb_jour_non_collecte  
8               0.0  93057R0052-99419                     0  
25              0.0  69383S3423-59032                     0  
26              0.0  69383S3423-59032                     0  
27              0.0  69383S3423-59032                     0  
28              0.0  69383S3423-59032                     0  
..              ...               ...                   ...  
691             0.0  69383S3423-59032                     0  
692             0.0  69383S3423-59032                     0  
693             0.0  69383S3423-59032                     0  
696             0.0  56114R0047-07924                     0  
697             0.0  59350R0631-00824                     0  

[124 rows x 16 columns]

In [41]:
df_Ecart_PRM_TrouRaquette = df_Ecart_PRM_TrouRaquette[df_Ecart_PRM_TrouRaquette.id_pdk_com.str[0:2]!='99']
df_Ecart_PRM_TrouRaquette = df_Ecart_PRM_TrouRaquette[df_Ecart_PRM_TrouRaquette.id_pdk_com.str[0:5]!='69993']


In [42]:
df_Ecart_PRM_TrouRaquette.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123 entries, 8 to 697
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id_c                     123 non-null    object 
 1   prm_dec                  123 non-null    object 
 2   prm_pose                 123 non-null    object 
 3   id_pdc                   123 non-null    object 
 4   dis_id                   123 non-null    int64  
 5   distributeur             123 non-null    object 
 6   prm_premiere_decouverte  123 non-null    object 
 7   Nb_Da                    98 non-null     float64
 8   dan_nni_ouverture        67 non-null     object 
 9   numero_da                98 non-null     object 
 10  nom_da                   98 non-null     object 
 11  statut_da                98 non-null     object 
 12  Nb_Da_Clos               123 non-null    float64
 13  Nb_Da_en_cours           123 non-null    float64
 14  id_pdk_com               1

In [43]:
# le .str accepte le type object de id_c mais j'ai qd même mis .astype(str) pour le transformer en str par précaution
df_Ecart_PRM_TrouRaquette = df_Ecart_PRM_TrouRaquette.assign(type_ec = df_Ecart_PRM_TrouRaquette.id_c.astype(str).str[0:2]+df_Ecart_PRM_TrouRaquette.id_c.astype(str).str[4:6])
df_Ecart_PRM_TrouRaquette

id_c         prm_dec        prm_pose          id_pdc  dis_id  \
8    021875130987  00000000000000  22554558557460  PDC00010497943       2   
25   062064347901  1053B29                1053B29  PDC00028516624       2   
26   062064347901  1053B29                1053B29  PDC00028516624       2   
27   062064347901  1053B29                1053B29  PDC00028516624       2   
28   062064347901  1053B29                1053B29  PDC00028516624       2   
..            ...             ...             ...             ...     ...   
691  062064347915  2146B06                2146B06  PDC00028514451       2   
692  062064347915  2146B06                2146B06  PDC00028514451       2   
693  062064347915  2146B06                2146B06  PDC00028514451       2   
696  042113000218  LGDO56114P0047  LGDO56114R0047  PDC00042587376       2   
697  021875042319  01106078079395  01105788643761  PDC00012518035       2   

    distributeur prm_premiere_decouverte  Nb_Da dan_nni_ouverture  \
8         ENEDIS          22554558557460    NaN               NaN   
25        ENEDIS          00000000000000    6.0            I09772   
26        ENEDIS          00000000000000    6.0            F17575   
27        ENEDIS          00000000000000    6.0            B28746   
28        ENEDIS          00000000000000    6.0            B28746   
..           ...                     ...    ...               ...   
691       ENEDIS          00000000000000    4.0            B28746   
692       ENEDIS          00000000000000    4.0          CMBAA65N   
693       ENEDIS          00000000000000    4.0              None   
696       ENEDIS          00000000000000    1.0              None   
697       ENEDIS          01105788643761    NaN               NaN   

          numero_da                 nom_da statut_da  Nb_Da_Clos  \
8               NaN                    NaN       NaN         0.0   
25   DA221114FLZWGU  Réco C - Écart de PRM      Clos         6.0   
26   DA221031FLUIEU  Réco C - Écart de PRM      Clos         6.0   
27   DA221017FLNRFV  Réco C - Écart de PRM      Clos         6.0   
28   DA211214FDVSUG  Réco C - Écart de PRM      Clos         6.0   
..              ...                    ...       ...         ...   
691  DA211214FDVSTV  Réco C - Écart de PRM      Clos         4.0   
692  DA211103FCKAQZ  Réco C - Écart de PRM      Clos         4.0   
693  DA200907EETTHZ  Réco C - Écart de PRM      Clos         4.0   
696  DA220912FKQSOA  Réco C - Écart de PRM      Clos         1.0   
697             NaN                    NaN       NaN         0.0   

     Nb_Da_en_cours        id_pdk_com  nb_jour_non_collecte type_ec  
8               0.0  93057R0052-99419                     0    0275  
25              0.0  69383S3423-59032                     0    0664  
26              0.0  69383S3423-59032                     0    0664  
27              0.0  69383S3423-59032                     0    0664  
28              0.0  69383S3423-59032                     0    0664  
..              ...               ...                   ...     ...  
691             0.0  69383S3423-59032                     0    0664  
692             0.0  69383S3423-59032                     0    0664  
693             0.0  69383S3423-59032                     0    0664  
696             0.0  56114R0047-07924                     0    0413  
697             0.0  59350R0631-00824                     0    0275  

[123 rows x 17 columns]

In [44]:
df_Ecart_PRM_TrouRaquette = df_Ecart_PRM_TrouRaquette[~df_Ecart_PRM_TrouRaquette.id_pdc.isin(['PDC00028515571','PDC00028513592','PDC00028515765','PDC00028515874','PDC00028515031','PDC00028516114',
                        'PDC00028514487','PDC00028516317','PDC00028513753','PDC00028514259','PDC00028513529','PDC00028515446',
                        'PDC00028516229','PDC00028514632','PDC00028514647','PDC00028515137',
                        'PDC00028514210','PDC00028513363','PDC00028513238','PDC00028514838','PDC00028516265','PDC00028513475',
                        'PDC00028515324','PDC00028514383','PDC00028514348','PDC00028517308','PDC00028515216','PDC00028514970',
                        'PDC00028516624','PDC00028516551','PDC00028515394','PDC00028518462',
                        'PDC00028516653','PDC00028514738','PDC00028514532','PDC00028516451','PDC00028515610','PDC00028516148',
                        'PDC00028513242','PDC00028515855','PDC00028516431','PDC00028515985','PDC00028514451','PDC00028514924',
                        'PDC00028515502','PDC00028515123','PDC00028513585','PDC00028516017','PDC00028515253'])].query("type_ec !='0202' and type_ec !='0413'")

In [45]:
df_Analyse_Ecart_PRM_NonResolu = df_Ecart_PRM_TrouRaquette.query("Nb_Da_Clos>=1 and Nb_Da_en_cours ==0").reset_index(drop=True)

In [46]:
Df_Analyse_Ecart_PRM_PostReco = df_Ecart_PRM_TrouRaquette.query("Nb_Da_Clos==0 and Nb_Da_en_cours==0").reset_index(drop=True)

In [47]:
import os 
directory = os.getcwd()
rep = directory+"/export_ecart_prm/"

fc.split_df_to_csv(df_Analyse_Ecart_PRM_NonResolu,999,rep,date_saisie,"df_Analyse_Ecart_PRM_NonResolu")
fc.split_df_to_csv(Df_Analyse_Ecart_PRM_PostReco,999,rep,date_saisie,"Df_Analyse_Ecart_PRM_PostReco")
fc.split_df_to_csv(df_eligibles,999,rep,date_saisie,"df_eligibles")

import shutil as sh
sh.make_archive('export_ecart_prm','zip',rep)

In [48]:
# zip du dossier
#import shutil as sh
#sh.make_archive('nom_dossier','zip',directory)